In [2]:
import requests
from google.transit import gtfs_realtime_pb2
from datetime import datetime
import os

def fetch_and_parse_feed(url):
    print(f"Fetching GTFS feed from: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        feed = gtfs_realtime_pb2.FeedMessage()
        feed.ParseFromString(response.content)
        return feed
    else:
        raise Exception(f"Failed to fetch GTFS feed: {response.status_code}")

# Get current timestamp for filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# URLs
trip_updates_url = "https://api.pugetsound.onebusaway.org/api/gtfs_realtime/trip-updates-for-agency/1.pb?key=org.onebusaway.iphone"
vehicle_positions_url = "https://api.pugetsound.onebusaway.org/api/gtfs_realtime/vehicle-positions-for-agency/1.pb?key=org.onebusaway.iphone"
alerts_url = "https://api.pugetsound.onebusaway.org/api/gtfs_realtime/alerts-for-agency/1.pb?key=org.onebusaway.iphone"


# Fetch and parse feeds
trip_updates = fetch_and_parse_feed(trip_updates_url)
vehicle_positions = fetch_and_parse_feed(vehicle_positions_url)
alerts = fetch_and_parse_feed(alerts_url)

# Create output folder (optional)
output_dir = "./gtfsRT-Analysis/gtfs_outputs"
os.makedirs(output_dir, exist_ok=True)

# Write decoded (human-readable) text files
with open(f"{output_dir}/trip_updates_{timestamp}.txt", "w", encoding='utf-8') as f:
    f.write(str(trip_updates))

with open(f"{output_dir}/vehicle_positions_{timestamp}.txt", "w", encoding='utf-8') as f:
    f.write(str(vehicle_positions))

with open(f"{output_dir}/alerts_{timestamp}.txt", "w", encoding='utf-8') as f:
    f.write(str(alerts))


Fetching GTFS feed from: https://api.pugetsound.onebusaway.org/api/gtfs_realtime/trip-updates-for-agency/1.pb?key=org.onebusaway.iphone
Fetching GTFS feed from: https://api.pugetsound.onebusaway.org/api/gtfs_realtime/vehicle-positions-for-agency/1.pb?key=org.onebusaway.iphone
Fetching GTFS feed from: https://api.pugetsound.onebusaway.org/api/gtfs_realtime/alerts-for-agency/1.pb?key=org.onebusaway.iphone


In [3]:
# Get current timestamp for filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# URLs
host = "localhost"  # Replace with actual host if needed
port = 1234  # Replace with actual port if needed
trip_updates_url = f"http://{host}:{port}/trip-updates"
vehicle_positions_url = f"http://{host}:{port}/vehicle-positions"
alerts_url = f"http://{host}:{port}/service-alerts"


# Fetch and parse feeds
trip_updates = fetch_and_parse_feed(trip_updates_url)
vehicle_positions = fetch_and_parse_feed(vehicle_positions_url)
alerts = fetch_and_parse_feed(alerts_url)

# Create output folder (optional)
output_dir = "./gtfsRT-Analysis/gtfs_outputs_mine"
os.makedirs(output_dir, exist_ok=True)

# Write decoded (human-readable) text files
with open(f"{output_dir}/trip_updates_{timestamp}.txt", "w", encoding='utf-8') as f:
    f.write(str(trip_updates))

with open(f"{output_dir}/vehicle_positions_{timestamp}.txt", "w", encoding='utf-8') as f:
    f.write(str(vehicle_positions))

with open(f"{output_dir}/alerts_{timestamp}.txt", "w", encoding='utf-8') as f:
    f.write(str(alerts))


Fetching GTFS feed from: http://localhost:1234/trip-updates
Fetching GTFS feed from: http://localhost:1234/vehicle-positions
Fetching GTFS feed from: http://localhost:1234/service-alerts


# Analyse

In [4]:
from datetime import datetime, timedelta

def print_feed_insights(feed, feed_name):
    print(f"\n--- {feed_name.upper()} ---")

    # Header timestamp (in seconds since epoch)
    if feed.header.timestamp:
        updated_time = datetime.fromtimestamp(feed.header.timestamp)
        age = datetime.now() - updated_time
        print(f"Updated at: {updated_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Updated {age.total_seconds() // 60:.1f} minutes ago")
    else:
        print("No timestamp found in header")

    # Entity count
    print(f"Total entities: {len(feed.entity)}")

    # Custom breakdowns
    if feed_name == "trip_updates":
        count_with_stop_updates = sum(1 for e in feed.entity if e.trip_update and e.trip_update.stop_time_update)
        print(f"Entities with stop time updates: {count_with_stop_updates}")
    elif feed_name == "vehicle_positions":
        vehicle_ids = [e.vehicle.vehicle.id for e in feed.entity if e.vehicle and e.vehicle.vehicle.id]
        print(f"Unique vehicle IDs: {len(set(vehicle_ids))}")
    elif feed_name == "alerts":
        alert_count = sum(1 for e in feed.entity if e.alert)
        print(f"Entities with alerts: {alert_count}")

# Print insights
print_feed_insights(trip_updates, "trip_updates")
print_feed_insights(vehicle_positions, "vehicle_positions")
print_feed_insights(alerts, "alerts")



--- TRIP_UPDATES ---
Updated at: 2025-05-03 22:10:00
Updated 112384.0 minutes ago
Total entities: 1
Entities with stop time updates: 0

--- VEHICLE_POSITIONS ---
Updated at: 2025-05-01 22:30:00
Updated 115244.0 minutes ago
Total entities: 100
Unique vehicle IDs: 100

--- ALERTS ---
Updated at: 2025-05-01 22:30:00
Updated 115244.0 minutes ago
Total entities: 25
Entities with alerts: 25
